In [2]:
import random

# Parameters

These define the fields in which we're doing all of our computations.

In [3]:
# small and large field
Q = 6497992661811505123 # < 64 bits
P = 1802216888453791673313287943102424579859887305661122324585863735744776691801009887 # < 270 bits

# Public elements

These will represent any value that we wish to use in our computation that is **not** required to be kept private -- in fact, we'll often assume that both parties know all public values. The reason for having these (as opposed to just using floats etc.) is that all values must be brought into our finite field before we can use them; this class takes care of doing that.

In [37]:
class PublicElement:
    
    def __init__(self, element):
        self.element = element
    
    @staticmethod
    def from_element(element):
        return PublicElement(element)
    
    def unwrap(self):
        return self.element
    
    def add(x, y):
        if type(y) is int: y = PublicElement(y)
        if type(y) is PublicElement:
            element = (x.element + y.element) % Q
            return PublicElement(element)
        if type(y) is PrivateElement:
            share0 = (x.element + y.share0) % Q
            share1 =              y.share1
            return PrivateElement(None, share0, share1)
        
    def sub(x, y):
        if type(y) is int: y = PublicElement(y)
        if type(y) is PublicElement:
            element = (x.element - y.element) % Q
            return PublicElement(element)
        if type(y) is PrivateElement:
            share0 = (x.element + Q - y.share0) % Q
            share1 = (            Q - y.share1) % Q
            return PrivateElement(None, share0, share1)
        
    def mul(x, y):
        if type(y) is int: y = PublicElement(y)
        if type(y) is PublicElement:
            element = (x.element * y.element) % Q
            return PublicElement(element)
        if type(y) is PrivateElement:
            share0 = (x.element * y.share0) % Q
            share1 = (x.element * y.share1) % Q
            return PrivateElement(None, share0, share1)
    
    def square(x):
        element = pow(x.element, 2, Q)
        return PublicElement(element)
    
#     def pows(x, highest_power):
#         x_powers = ( np.power(x.values, e) % Q for e in range(0, highest_power+1) )
#         return [ PublicTensor(v) for v in x_powers ]
    
    def __add__(x, y):
        return x.add(y)
    
    def __sub__(x, y):
        return x.sub(y)
    
    def __mul__(x, y):
        return x.mul(y)
        
    def __repr__(self):
        return "PublicElement(%s)" % self.unwrap()

In [38]:
x = PublicElement(5)
y = PublicElement(7)

z = x - y
print(z)

PublicElement(6497992661811505121)


In [53]:
def encode_integer(value):
    return value % Q

def decode_integer(element):
    return element if element <= Q//2 else element - Q

class PublicInteger:
    
    def __init__(self, integer, element=None):
        if integer is not None:
            element = integer # TODO encode
        self.element = element

    @staticmethod
    def from_element(element):
        return PublicInteger(None, element)

    @staticmethod
    def from_integer(integer):
        return PublicInteger(integer)

    def unwrap(self):
        return decode_integer(self.element)

    def add(x, y):
        if type(y) is int: y = PublicInteger(y)
        if type(y) is float: y = PublicFixedpoint(y)
        if type(y) is PublicInteger:
            element = (x.element + y.element) % Q
            return PublicInteger.from_element(element)
        if type(y) is PrivateInteger:
            share0 = (x.element + y.share0) % Q
            share1 =              y.share1
            return PrivateInteger(None, share0, share1)

    def sub(x, y):
        if type(y) is int: y = PublicInteger(y)
        if type(y) is float: y = PublicFixedpoint(y)
        if type(y) is PublicInteger:
            element = (x.element - y.element) % Q
            return PublicInteger.from_element(element)
        if type(y) is PrivateInteger:
            share0 = (x.element + Q - y.share0) % Q
            share1 = (            Q - y.share1) % Q
            return PrivateInteger(None, share0, share1)

    def mul(x, y):
        if type(y) is int: y = PublicInteger(y)
        if type(y) is float: y = PublicFixedpoint(y)
        if type(y) is PublicInteger:
            element = (x.element * y.element) % Q
            return PublicInteger.from_element(element)
        if type(y) is PrivateInteger:
            share0 = (x.element * y.share0) % Q
            share1 = (x.element * y.share1) % Q
            return PrivateInteger(None, share0, share1)

    def square(x):
        element = pow(x.element, 2, Q)
        return PublicInteger(element)

    def __add__(x, y):
        return x.add(y)

    def __sub__(x, y):
        return x.sub(y)

    def __mul__(x, y):
        return x.mul(y)

    def __repr__(self):
        return "PublicInteger(%s)" % self.unwrap()

In [61]:
class PublicElement:
    
    def __init__(self, element):
        self.element = element
    
    @staticmethod
    def from_element(element):
        return PublicElement(element)
    
    def unwrap(self):
        return self.element
    
    def add(x, y):
        if isinstance(y, PublicElement):
            element = (x.element + y.element) % Q
            return PublicElement(element)
        if isinstance(y, PrivateElement):
            share0 = (x.element + y.share0) % Q
            share1 =              y.share1
            return PrivateElement(None, share0, share1)
        raise TypeError("{} add {}" % (type(x), type(y)))
        
    def sub(x, y):
        if isinstance(y, PublicElement):
            element = (x.element - y.element) % Q
            return PublicElement(element)
        if isinstance(y, PrivateElement):
            share0 = (x.element + Q - y.share0) % Q
            share1 = (            Q - y.share1) % Q
            return PrivateElement(None, share0, share1)
        raise TypeError("{} sub {}" % (type(x), type(y)))
        
    def mul(x, y):
        if isinstance(y, PublicElement):
            element = (x.element * y.element) % Q
            return PublicElement(element)
        if isinstance(y, PrivateElement):
            share0 = (x.element * y.share0) % Q
            share1 = (x.element * y.share1) % Q
            return PrivateElement(None, share0, share1)
        raise TypeError("{} mul {}" % (type(x), type(y)))
    
    def square(x):
        element = pow(x.element, 2, Q)
        return PublicElement(element)
    
#     def pows(x, highest_power):
#         x_powers = ( np.power(x.values, e) % Q for e in range(0, highest_power+1) )
#         return [ PublicTensor(v) for v in x_powers ]
    
    def __add__(x, y):
        return x.add(y)
    
    def __sub__(x, y):
        return x.sub(y)
    
    def __mul__(x, y):
        return x.mul(y)
        
    def __repr__(self):
        return "PublicElement(%s)" % self.unwrap()

In [59]:
def encode_integer(value):
    return value % Q

def decode_integer(element):
    return element if element <= Q//2 else element - Q

def convert(x, y):    
    if type(y) is int: y = PublicInteger.from_value(y)
    if type(y) is float: y = PublicFixedpoint.from_value(y)
    
    
    
    


class PublicInteger(PublicElement):
    
    def __init__(self, integer, element=None):
        if integer is not None:
            element = encode_integer(integer)
        super().__init__(element)

    @staticmethod
    def from_element(element):
        return PublicInteger(None, element)

    @staticmethod
    def from_integer(integer):
        return PublicInteger(integer)

    def unwrap(self):
        return decode_integer(super().unwrap())

    def wrap_if_needed(self, value):
        if type(value) is int: value = PublicInteger(value)
        if type(value) is float: value = PublicFixedpoint(value)
        return value
    
    def add(x, y):
        y = x.wrap_if_needed(y)
        z = super().add(x, y)
        return z.
        
        if type(y) is PublicInteger:
            element = (x.element + y.element) % Q
            return PublicInteger.from_element(element)
        if type(y) is PrivateInteger:
            share0 = (x.element + y.share0) % Q
            share1 =              y.share1
            return PrivateInteger(None, share0, share1)

    def sub(x, y):
        y = x.wrap_if_needed(y)
        if type(y) is PublicInteger:
            element = (x.element - y.element) % Q
            return PublicInteger.from_element(element)
        if type(y) is PrivateInteger:
            share0 = (x.element + Q - y.share0) % Q
            share1 = (            Q - y.share1) % Q
            return PrivateInteger(None, share0, share1)

    def mul(x, y):
        y = x.wrap_if_needed(y)
        if type(y) is PublicInteger:
            element = (x.element * y.element) % Q
            return PublicInteger.from_element(element)
        if type(y) is PrivateInteger:
            share0 = (x.element * y.share0) % Q
            share1 = (x.element * y.share1) % Q
            return PrivateInteger(None, share0, share1)

    def __repr__(self):
        return "PublicInteger(%s)" % self.unwrap()

In [60]:
x = PublicInteger(5)
y = PublicInteger(7)

z = x - y
print(z)

PublicInteger(-2)


# Private element

These represent any value that we wish to keep private.

In [ ]:
def share(secret):
    share0 = random.randrange(Q)
    share1 = (secret - share0) % Q
    return [share0, share1]

def reconstruct(share0, share1):
    return (share0 + share1) % Q

In [ ]:
def generate_mul_triple():
    a = random.randrange(Q)
    b = random.randrange(Q)
    c = (a * b) % Q
    return PrivateElement(a), PrivateElement(b), PrivateElement(c)

In [ ]:
def generate_square_triple():
    a = random.randrange(Q)
    b = pow(a, 2, Q)
    return PrivateElement(a), PrivateElement(b)

In [10]:
class PrivateElement:
    
    def __init__(self, element, share0=None, share1=None):
        if not element is None:
            share0, share1 = share(element)
        self.share0 = share0
        self.share1 = share1
    
    def reconstruct(self):
        return PublicElement(reconstruct(self.share0, self.share1))
    
    def unwrap(self):
        return reconstruct(self.share0, self.share1)
    
    def add(x, y):
        if type(y) is int: y = PublicElement(y)
        if type(y) is PublicElement:
            share0 = (x.share0 + y.element) % Q
            share1 =  x.share1
            return PrivateElement(None, share0, share1)
        if type(y) is PrivateElement:
            share0 = (x.share0 + y.share0) % Q
            share1 = (x.share1 + y.share1) % Q
            return PrivateElement(None, share0, share1)
        
    def sub(x, y):
        if type(y) is int: y = PublicElement(y)
        if type(y) is PublicElement:
            share0 = (x.share0 - y.element) % Q
            share1 =  x.share1
            return PrivateElement(None, share0, share1)
        if type(y) is PrivateElement:
            share0 = (x.share0 - y.share0) % Q
            share1 = (x.share1 - y.share1) % Q
            return PrivateElement(None, share0, share1)
        
    def mul(x, y):
        if type(y) is int: y = PublicElement(y)
        if type(y) is PublicElement:
            share0 = (x.share0 * y.element) % Q
            share1 = (x.share1 * y.element) % Q
            return PrivateElement(None, share0, share1)
        if type(y) is PrivateElement:
            a, b, a_mul_b = generate_mul_triple()
            alpha = (x - a).reconstruct()
            beta  = (y - b).reconstruct()
            return alpha.mul(beta) + \
                   alpha.mul(b) + \
                   a.mul(beta) + \
                   a_mul_b
                    
    def square(x):
        a, aa = generate_square_triple()
        alpha = (x - a).reconstruct()
        return alpha.square() + \
               (a * alpha) * 2 + \
               aa
    
#     def pows(x, highest_power):
#         x_powers = ( np.power(x.values, e) % Q for e in range(0, highest_power+1) )
#         return [ PublicTensor(v) for v in x_powers ]
    
    def __add__(x, y):
        return x.add(y)
    
    def __sub__(x, y):
        return x.sub(y)
    
    def __mul__(x, y):
        return x.mul(y)
        
    def __repr__(self):
        return "PrivateElement(%s)" % self.unwrap()

# Tests

In [ ]:
v = 5
w = 3

for x_type in [PublicElement, PrivateElement]:
    for y_type in [PublicElement, PrivateElement]:
        
        x = x_type(v)
        y = y_type(w)

        z = x + y; assert z.unwrap() == v + w
        z = x - y; assert z.unwrap() == v - w
        z = x * y; assert z.unwrap() == v * w
        z = x.square(); assert z.unwrap() == v * v

## Powering and polynomials

In [ ]:
def generate_powering_triple(exponent, field=Q):
    a = random.randrange(field)
    return [ share(pow(a, e, field)) for e in range(1, exponent+1) ]

a, aa, aaa, aaaa = generate_powering_triple(4)
assert reconstruct(a) * reconstruct(a) % Q == reconstruct(aa) 
assert reconstruct(aa) * reconstruct(a) % Q == reconstruct(aaa)
assert reconstruct(aaa) * reconstruct(a) % Q == reconstruct(aaaa)

In [ ]:
from functools import reduce
from scipy.misc import comb
binom = lambda n, k: comb(n, k, exact=True)

ONE = [1,0] # constant sharing of 1

def pows(x, triple, field=Q):
    # local masking
    a = triple[0]
    v = sub(x, a)
    # communication: the players simultanously send their share to the other
    epsilon = reconstruct(v)
    # local combination
    x_powers = []
    for exponent in range(1, len(triple)+1):
        # prepare all term values
        a_powers = [ONE] + triple[:exponent]
        e_powers = [ pow(epsilon, e, Q) for e in range(exponent+1) ]
        coeffs   = [ binom(exponent, k) for k in range(exponent+1) ]
        # compute and sum terms
        terms = ( mul_public(a, c * e) for a, c, e in zip(a_powers, coeffs, reversed(e_powers)) )
        x_powers.append(reduce(lambda x,y: add(x, y, field), terms))
    return x_powers

x = share(4)

xs = pows(x, generate_powering_triple(3))
assert [ reconstruct(x) for x in xs ] == [ pow(4,e,Q) for e in range(1,3+1) ]

xs = pows(x, generate_powering_triple(10))
assert [ reconstruct(x) for x in xs ] == [ pow(4,e,Q) for e in range(1,10+1) ]

In [ ]:
def pol_public(x, coeffs, triple, field):
    powers = [ONE] + pows(x, triple, field)
    terms = ( mul_public(xe, ce, field) for xe,ce in zip(powers, coeffs) )
    return reduce(lambda y,z: add(y, z, field), terms)

x = share(5)
coeffs = [1,2,3,4]
y = pol_public(x, coeffs, generate_powering_triple(len(coeffs)-1, Q), Q)
assert reconstruct(y) == (1*pow(5,0) + 2*pow(5,1) + 3*pow(5,2) + 4*pow(5,3)) % Q

### Up and down sharing

In [ ]:
def generate_statistical_mask():
    return random.randrange(2*BOUND * BASE**KAPPA)

def generate_zero_triple(field):
    return share(0, field)

def convert(x, from_field, to_field, zero_triple):
    # local mapping to positive representation
    x = add_public(x, BOUND, from_field)
    # local masking and conversion by player 0
    r = generate_statistical_mask()
    y0 = (zero_triple[0] - r) % to_field
    # exchange of masked share: one round of communication
    e = (x[0] + r) % from_field
    # local conversion by player 1
    xr = (e + x[1]) % from_field
    y1 = (zero_triple[1] + xr) % to_field
    # local combine
    y = [y0, y1]
    # local mapping back from positive representation
    y = sub_public(y, BOUND, to_field)
    return y

def upshare(x, large_zero_triple):
    return convert(x, Q, P, large_zero_triple)

def downshare(x, small_zero_triple):
    return convert(x, P, Q, small_zero_triple)

x = share(encode(-.5, Q), Q)
y = upshare(x, generate_zero_triple(P))
assert reconstruct(y, P) == encode(-.5, P)
z = downshare(y, generate_zero_triple(Q))
assert reconstruct(z, Q) == encode(-.5, Q)

In [ ]:
x = share(encode(-.5, Q), Q)
x2 = mul(x, x, generate_multiplication_triple(Q), Q)
x4 = mul(x2, x2, generate_multiplication_triple(Q), Q)
y = truncate(x4, 4 * PRECISION_FRACTIONAL - PRECISION_FRACTIONAL, Q)
assert not decode(reconstruct(y, Q), Q) == (-.5)**4

x = share(encode(-.5, Q), Q)
x = upshare(x, generate_zero_triple(P))
x2 = mul(x, x, generate_multiplication_triple(P), P)
x4 = mul(x2, x2, generate_multiplication_triple(P), P)
y = truncate(x4, 4 * PRECISION_FRACTIONAL - PRECISION_FRACTIONAL, P)
y = downshare(y, generate_zero_triple(Q))
assert decode(reconstruct(y, Q), Q) == (-.5)**4

## Fixed-point encoding

In [ ]:


KAPPA = 9 # ~29 bits

PRECISION_INTEGRAL = 2
PRECISION_FRACTIONAL = 7
PRECISION = PRECISION_INTEGRAL + PRECISION_FRACTIONAL
BOUND = BASE**PRECISION

Q_MAXDEGREE = 2
assert Q > BASE**(PRECISION * Q_MAXDEGREE) # supported multiplication degree (without truncation)
assert Q > 2*BOUND * BASE**KAPPA # supported kappa when in positive range 

P_MAXDEGREE = 9
assert P > Q
assert P > BASE**(PRECISION * P_MAXDEGREE)

In [ ]:
BASE = 2

def encode(rational):
    upscaled = int(rational * BASE**PRECISION_FRACTIONAL)
    field_element = upscaled % Q
    return field_element

def decode(field_element):
    upscaled = field_element if field_element <= Q//2 else field_element - Q
    rational = upscaled / BASE**PRECISION_FRACTIONAL
    return rational

In [ ]:
# using trick from SecureML paper that only requires a local operation

def truncate(x, amount=PRECISION_FRACTIONAL, field=Q):
    y0 = x[0] // BASE**amount
    y1 = field - ((field - x[1]) // BASE**amount)
    return [y0, y1]

In [ ]:
x = share(encode(-.5))

x2 = truncate(mul( x,  x, generate_mul_triple()), PRECISION_FRACTIONAL)
x4 = truncate(mul(x2, x2, generate_mul_triple()), PRECISION_FRACTIONAL)
y = decode(reconstruct(x4))
assert y == (-.5)**4

x2 = mul( x,  x, generate_mul_triple())
x4 = mul(x2, x2, generate_mul_triple())
y = decode(reconstruct(truncate(x4, 2*PRECISION_FRACTIONAL)))
assert not y == (-.5)**4

In [ ]:
def encode(rationals):
    return (rationals * BASE**PRECISION_FRACTIONAL).astype('int').astype(DTYPE) % Q

def decode(elements):
    map_negative_range = np.vectorize(lambda element: element if element <= Q/2 else element - Q)
    return map_negative_range(elements) / BASE**PRECISION_FRACTIONAL

In [ ]:
class PublicFixedpointValue(PublicValue):
    
    def __init__(self, value):
        self.value = value
    
    def unwrap(self):
        return decode(self.value)
    
    def wrap_if_needed(x):
        if type(y) is int:
            y = PublicFixedpointValue(y)
        if type(y) is float:
            y = PublicFixedpointValue(y)
        return y
        
    def mul(x, y):
        if type(y) is int: y = PublicValue(y)
        if type(y) is PublicValue:
            value = (x.value * y.value) % Q
            return PublicValue(value)
        if type(y) is PrivateValue:
            share0 = (x.value * y.share0) % Q
            share1 = (x.value * y.share1) % Q
            return PrivateValue(None, share0, share1)
    
    def square(x):
        value = pow(x.value, 2, Q)
        return PublicValue(value)
    
#     def pows(x, highest_power):
#         x_powers = ( np.power(x.values, e) % Q for e in range(0, highest_power+1) )
#         return [ PublicTensor(v) for v in x_powers ]
    
    def __add__(x, y):
        return x.add(y)
    
    def __sub__(x, y):
        return x.sub(y)
    
    def __mul__(x, y):
        return x.mul(y)
        
    def __repr__(self):
        return "PublicValue(%s)" % self.value

In [ ]:
class PublicValue:
    
    def __init__(self, value):
        self.value = value
    
    def unwrap(self):
        return self.value
    
    def wrap_if_needed(y):
        if type(y) is int: y = PublicValue(y)
        return y
    
    def add(x, y):
        y = self.wrap_if_needed(y)
        if isinstance(y, PublicValue):
            value = (x.value + y.value) % Q
            return self.__init__(value) ????
        if isinstance(y, PrivateValue):
            share0 = (x.value + y.share0) % Q
            share1 =            y.share1
            return PrivateValue(None, share0, share1)
        
    def sub(x, y):
        if type(y) is int: y = PublicValue(y)
        if type(y) is PublicValue:
            value = (x.value - y.value) % Q
            return PublicValue(value)
        if type(y) is PrivateValue:
            share0 = (x.value + Q - y.share0) % Q
            share1 = (          Q - y.share1) % Q
            return PrivateValue(None, share0, share1)
        
    def mul(x, y):
        if type(y) is int: y = PublicValue(y)
        if type(y) is PublicValue:
            value = (x.value * y.value) % Q
            return PublicValue(value)
        if type(y) is PrivateValue:
            share0 = (x.value * y.share0) % Q
            share1 = (x.value * y.share1) % Q
            return PrivateValue(None, share0, share1)
    
    def square(x):
        value = pow(x.value, 2, Q)
        return PublicValue(value)
    
#     def pows(x, highest_power):
#         x_powers = ( np.power(x.values, e) % Q for e in range(0, highest_power+1) )
#         return [ PublicTensor(v) for v in x_powers ]
    
    def __add__(x, y):
        return x.add(y)
    
    def __sub__(x, y):
        return x.sub(y)
    
    def __mul__(x, y):
        return x.mul(y)
        
    def __repr__(self):
        return "PublicValue(%s)" % self.value